In [12]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the Flowers-102 dataset
(ds_train, ds_test), ds_info = tfds.load('tf_flowers', split=['train[:70%]', 'train[70%:]'], as_supervised=True, with_info=True)

# Define a function to resize images
def resize_image(image, label):
    image = tf.image.resize(image, (32, 32))
    return image, label

# Apply resizing to the dataset
ds_train = ds_train.map(resize_image)
ds_test = ds_test.map(resize_image)

# Normalize pixel values to be in the range [0, 1]
ds_train = ds_train.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))
ds_test = ds_test.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))

# Shuffle and batch the datasets
ds_train = ds_train.shuffle(10000).batch(64)
ds_test = ds_test.batch(64)

# Convert labels to categorical format
def categorical_labels(images, labels):
    return images, tf.one_hot(labels, depth=102)  # 102 classes in Flowers-102 dataset

ds_train = ds_train.map(categorical_labels)
ds_test = ds_test.map(categorical_labels)

# Verify the shapes
for images, labels in ds_train.take(1):
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)

for images, labels in ds_test.take(1):
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)


Images shape: (64, 32, 32, 3)
Labels shape: (64, 102)
Images shape: (64, 32, 32, 3)
Labels shape: (64, 102)


In [35]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# Create the ResNet model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    
# Add the top layers for classification
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(102, activation='softmax'))  #
#validation_split = 0.2
#validation_samples = int(validation_split * len(ds_train))
#training_samples = len(ds_train) - validation_samples
#ds_validation = ds_train.take(validation_samples)
#ds_train = ds_train.skip(validation_samples)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=15, verbose=1)
model.fit(ds_train, batch_size=32, epochs=100, validation_data=ds_validation, callbacks=[early_stopping])

Epoch 1/100
33/33 [==============================] - 10s 76ms/step - loss: 2.3576 - accuracy: 0.4511 - val_loss: 15.5359 - val_accuracy: 0.2637
Epoch 2/100
33/33 [==============================] - 2s 46ms/step - loss: 1.1529 - accuracy: 0.6466 - val_loss: 33.4096 - val_accuracy: 0.2266
Epoch 3/100
33/33 [==============================] - 1s 43ms/step - loss: 0.8990 - accuracy: 0.7122 - val_loss: 8.6839 - val_accuracy: 0.2344
Epoch 4/100
33/33 [==============================] - 1s 44ms/step - loss: 0.6763 - accuracy: 0.7764 - val_loss: 3.8584 - val_accuracy: 0.2480
Epoch 5/100
33/33 [==============================] - 2s 45ms/step - loss: 0.5794 - accuracy: 0.7929 - val_loss: 3.2514 - val_accuracy: 0.2559
Epoch 6/100
33/33 [==============================] - 1s 44ms/step - loss: 0.3742 - accuracy: 0.8741 - val_loss: 2.0523 - val_accuracy: 0.2422
Epoch 7/100
33/33 [==============================] - 1s 43ms/step - loss: 0.5768 - accuracy: 0.8216 - val_loss: 231.9373 - val_accuracy: 0.2695
E

33/33 [==============================] - 1s 42ms/step - loss: 1.0168 - accuracy: 0.6563 - val_loss: 51.4755 - val_accuracy: 0.3477
Epoch 59/100
33/33 [==============================] - 2s 45ms/step - loss: 0.8793 - accuracy: 0.6957 - val_loss: 10.6172 - val_accuracy: 0.5957
Epoch 60/100
33/33 [==============================] - 2s 46ms/step - loss: 0.8340 - accuracy: 0.7195 - val_loss: 7.1265 - val_accuracy: 0.6523
Epoch 61/100
33/33 [==============================] - 1s 44ms/step - loss: 0.7311 - accuracy: 0.7409 - val_loss: 0.8740 - val_accuracy: 0.6758
Epoch 62/100
33/33 [==============================] - 2s 46ms/step - loss: 0.6941 - accuracy: 0.7603 - val_loss: 0.5606 - val_accuracy: 0.7949
Epoch 63/100
33/33 [==============================] - 1s 43ms/step - loss: 0.6810 - accuracy: 0.7798 - val_loss: 0.6801 - val_accuracy: 0.7520
Epoch 63: early stopping


In [36]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Make predictions on the test dataset
predictions = model.predict(ds_test)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Manually generate ground truth labels for evaluation
true_labels = []
for _, label in ds_test:
    true_labels.extend(label.numpy())

true_labels = np.argmax(true_labels, axis=1)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Assuming you have test_loss and test_accuracy available
# If not, you'll need to calculate them based on your model and test dataset
print(f"Test Accuracy: {accuracy}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")


18/18 [==============================] - 1s 16ms/step
Test Accuracy: 0.6248864668483197
Accuracy: 0.6248864668483197
F1 Score: 0.6154539390263934


In [ ]:
from tensorflow import keras
model = keras.models.load_model(r"C:\Users\shaif\Downloads\Flowers102_32_model.h5")

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
    
num_classes = 102  # Update with the actual number of classes in your target data
x = model.layers[-4].output  # Access the last 4th layer from the end
x = GlobalAveragePooling2D()(x)
output = Dense(num_classes, activation='softmax')(x)

# Create the new model with the updated head
new_model = keras.models.Model(inputs=model.input, outputs=output)
early_stopping = EarlyStopping(monitor='val_loss', patience=15, verbose=1)
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
new_model.fit(ds_train, batch_size=32, epochs=100, validation_data=ds_validation, callbacks=[early_stopping])

Epoch 1/100
33/33 [==============================] - 14s 106ms/step - loss: 2.1945 - accuracy: 0.4511 - val_loss: 1902.0093 - val_accuracy: 0.1855
Epoch 2/100
33/33 [==============================] - 2s 72ms/step - loss: 1.2157 - accuracy: 0.6038 - val_loss: 82.7240 - val_accuracy: 0.2695
Epoch 3/100
33/33 [==============================] - 2s 53ms/step - loss: 0.9306 - accuracy: 0.6811 - val_loss: 2.6218 - val_accuracy: 0.4609
Epoch 4/100
33/33 [==============================] - 2s 57ms/step - loss: 0.8103 - accuracy: 0.7282 - val_loss: 66.6547 - val_accuracy: 0.2930
Epoch 5/100
33/33 [==============================] - 2s 52ms/step - loss: 0.6453 - accuracy: 0.7822 - val_loss: 6.9530 - val_accuracy: 0.6816
Epoch 6/100
33/33 [==============================] - 2s 53ms/step - loss: 0.5250 - accuracy: 0.8362 - val_loss: 0.9697 - val_accuracy: 0.8008
Epoch 7/100
33/33 [==============================] - 2s 51ms/step - loss: 0.3414 - accuracy: 0.8814 - val_loss: 0.4279 - val_accuracy: 0.8730

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import numpy as np

# Make predictions on the test dataset
predictions = new_model.predict(ds_test)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Manually generate ground truth labels for evaluation
true_labels = []
for _, label in ds_test:
    true_labels.extend(label.numpy())

true_labels = np.argmax(true_labels, axis=1)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Assuming you have test_loss and test_accuracy available
# If not, you'll need to calculate them based on your model and test dataset
print(f"Test Accuracy: {accuracy}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
